In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,449 kB]
Get:14 http://archive.ubuntu.com/u

In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import files

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://www.kristeligt-dagblad.dk/')


wait = WebDriverWait(driver, 10)

# accept cookies
try:
    accept_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.coi-banner__accept')))
    accept_button.click()
    print("Cookies accepted.")
except Exception as e:
    print(f"No cookies banner or could not accept cookies: {e}")

time.sleep(2)

try:
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#userProfile"]')))
    driver.execute_script("arguments[0].click();", login_button)  # Use JS click to ensure it's clicked
    print("Login button clicked.")
except Exception as e:
    print(f"Error clicking login button: {e}")

time.sleep(2)

try:
    email_input = wait.until(EC.visibility_of_element_located((By.ID, 'login_email')))
    password_input = driver.find_element(By.ID, 'login_pw')

    driver.execute_script("arguments[0].scrollIntoView(true);", email_input)

    # Use JavaScript to set the values if Selenium click doesn't work
    driver.execute_script("arguments[0].value = arguments[1];", email_input, 'grp613@alumni.ku.dk')
    driver.execute_script("arguments[0].value = arguments[1];", password_input, 'xxxx')

    print("Filled in login form using JavaScript.")

    login_submit_button = driver.find_element(By.CSS_SELECTOR, 'button.login-submit')

    driver.execute_script("arguments[0].scrollIntoView(true);", login_submit_button)

    actions = ActionChains(driver)
    actions.move_to_element(login_submit_button).click().perform()

    print("Login form submitted using ActionChains.")

    time.sleep(5)
except Exception as e:
    print(f"Error filling in login form or clicking submit: {e}")

base_url = "https://www.kristeligt-dagblad.dk/soeg?soeg={term}&page={page}"


all_articles = []
MAX_ARTICLES = 5000  # had maximum in the beginning so didnt scrape everything only to find out it was wrongly scraped

def scrape_kristeligt_articles(search_term):
    page = 0
    article_count = 0

    while article_count < MAX_ARTICLES:
        print(f"Scraping page {page} for search term: {search_term}")
        search_url = base_url.format(term=search_term, page=page)

        driver.get(search_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        articles = soup.find_all('article', class_='simple-teaser')
        if not articles:
            print(f"No more articles found for search term: {search_term} on page {page}. Stopping...")
            break

        for article in articles:
            if article_count >= MAX_ARTICLES:
                break

            article_url = article.find('a')['href']
            article_url = "https://www.kristeligt-dagblad.dk" + article_url
            driver.get(article_url)
            time.sleep(10)

            article_soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                title = article_soup.find('h1', class_='article__title').get_text()
            except:
                title = 'No title'

            try:
                subtitle = article_soup.find('p', class_='article__lead-para').get_text()
            except:
                subtitle = 'No subtitle'

            try:
                meta_items = article_soup.find_all('span', class_='article__meta-item')


                for item in meta_items:

                    if any(char.isdigit() for char in item.get_text()):
                        date = item.get_text()
                        break
                else:
                    date = 'No date'
            except:
                date = 'No date'

            try:
                content_div = article_soup.find('div', class_='article__content rich-text article__content--drop-cap')
                paragraphs = content_div.find_all('p')
                content = " ".join([p.get_text() for p in paragraphs])
            except:
                content = 'No content'

            all_articles.append({
                'search_term': search_term,
                'url': article_url,
                'title': title,
                'subtitle': subtitle,
                'date': date,
                'content': content,
                'search_term': term
            })

            article_count += 1

        page += 1


search_terms = ['facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

# through each term
for term in search_terms:
    scrape_kristeligt_articles(term)

df = pd.DataFrame(all_articles)

df.to_csv('scraped_kristeligt_articles.csv', index=False)
files.download('scraped_kristeligt_articles.csv')

print(df.head())

driver.quit()



Cookies accepted.
Login button clicked.
Filled in login form using JavaScript.
Error filling in login form or clicking submit: Message: javascript error: {"status":60,"value":"[object HTMLButtonElement] has no size and location"}
  (Session info: chrome=129.0.6668.100)
Stacktrace:
#0 0x5b0a9fb6bb9a <unknown>
#1 0x5b0a9f851670 <unknown>
#2 0x5b0a9f8574a3 <unknown>
#3 0x5b0a9f859165 <unknown>
#4 0x5b0a9f8591ec <unknown>
#5 0x5b0a9f8a6e2e <unknown>
#6 0x5b0a9f8a6246 <unknown>
#7 0x5b0a9f8ed287 <unknown>
#8 0x5b0a9f8c5b82 <unknown>
#9 0x5b0a9f8e4dd6 <unknown>
#10 0x5b0a9f8c5923 <unknown>
#11 0x5b0a9f8936e7 <unknown>
#12 0x5b0a9f8946de <unknown>
#13 0x5b0a9fb3566b <unknown>
#14 0x5b0a9fb39611 <unknown>
#15 0x5b0a9fb214e5 <unknown>
#16 0x5b0a9fb3a192 <unknown>
#17 0x5b0a9fb066ef <unknown>
#18 0x5b0a9fb5a9d8 <unknown>
#19 0x5b0a9fb5aba7 <unknown>
#20 0x5b0a9fb6a9ec <unknown>
#21 0x7e89c0847ac3 <unknown>

Scraping page 0 for search term: facial recognition
Scraping page 1 for search term: faci

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

          search_term                                                url  \
0  facial recognition  https://www.kristeligt-dagblad.dk/danmark/torsdag   
1    face recognition  https://www.kristeligt-dagblad.dk/udland/dokum...   
2    face recognition  https://www.kristeligt-dagblad.dk/udland/tekst...   
3    face recognition  https://www.kristeligt-dagblad.dk/kirke-tro/ta...   
4       ansigtsaftryk  https://www.kristeligt-dagblad.dk/kronik/et-g%...   

                                               title  \
0  Restriktioner taber til religiøs frihed i USA,...   
1  Dokumentation: Læs Camerons tale om Storbritan...   
2                                 Tekst: Obamas tale   
3                        Talen, som paven ikke holdt   
4                          Et gådefuldt spejlbillede   

                                            subtitle                date  \
0  \n            Her samler Kristeligt Dagblad no...  17.12.20 Kl. 08:45   
1  \n            I sin længe ventede tale om Stor...  

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Thesis/Data/scraped_kristeligt_articles.csv')

In [ ]:
df = df.drop_duplicates(subset=['url'])

In [ ]:
df['merged_text'] = df['title'] + ' ' + df['subtitle'] + ' ' + df['content']

In [ ]:
df = df.drop(columns=['cleaned_date', 'date'])


In [ ]:

df = df.rename(columns={'standard_date': 'date', 'title': 'header', 'subtitle': 'subheader'})

In [ ]:
# Save the DataFrame to a CSV file
from google.colab import files

df.to_csv('KD_scraped_articles_clean_otherterms.csv', index=False)

files.download('KD_scraped_articles_clean_otherterms.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/KD/KD_scraped_articles_clean_otherterms.csv')
df1 = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/KD/KD_scraped_articles_clean.csv')
df2 = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/KD/scraped_kristeligt_articles.csv')

In [ ]:
df = pd.concat([df, df1, df2], ignore_index=True)
df

,search_term,url,header,subheader,content,merged_text,date
0,facial recognition,https://www.kristeligt-dagblad.dk/danmark/torsdag,"Restriktioner taber til religiøs frihed i USA,...",\n Her samler Kristeligt Dagblad no...,Morgensamling er Kristeligt Dagblads nyhedsove...,"Restriktioner taber til religiøs frihed i USA,...",2020-12-17
1,face recognition,https://www.kristeligt-dagblad.dk/udland/dokum...,Dokumentation: Læs Camerons tale om Storbritan...,\n I sin længe ventede tale om Stor...,This morning I want to talk about the future o...,Dokumentation: Læs Camerons tale om Storbritan...,2013-01-23
2,face recognition,https://www.kristeligt-dagblad.dk/udland/tekst...,Tekst: Obamas tale,\n Læs hele præsident Barack Obamas...,"Her er en udskrift af den tale, som Barack Oba...",Tekst: Obamas tale \n Læs hele præs...,2009-01-20
3,face recognition,https://www.kristeligt-dagblad.dk/kirke-tro/ta...,"Talen, som paven ikke holdt",\n Pave Benedikt XVI aflyste denne ...,It is a great joy for me to meet the community...,"Talen, som paven ikke holdt \n Pave...",2008-01-18
4,ansigtsaftryk,https://www.kristeligt-dagblad.dk/kronik/et-g%...,Et gådefuldt spejlbillede,\n Er ligklædet i Torino et håndgri...,Det er under alle omstændigheder vigtigt at hu...,Et gådefuldt spejlbillede \n Er lig...,2000-06-16
...,...,...,...,...,...,...,...
237,face recognition,https://www.kristeligt-dagblad.dk/udland/tekst...,Tekst: Obamas tale,\n Læs hele præsident Barack Obamas...,"Her er en udskrift af den tale, som Barack Oba...",Tekst: Obamas tale \n Læs hele præs...,20-01-09
238,face recognition,https://www.kristeligt-dagblad.dk/kirke-tro/ta...,"Talen, som paven ikke holdt",\n Pave Benedikt XVI aflyste denne ...,It is a great joy for me to meet the community...,"Talen, som paven ikke holdt \n Pave...",18-01-08
239,face recognition,https://www.kristeligt-dagblad.dk/udland/dokum...,Dokumentation: Læs Camerons tale om Storbritan...,\n I sin længe ventede tale om Stor...,This morning I want to talk about the future o...,Dokumentation: Læs Camerons tale om Storbritan...,23-01-13
240,face recognition,https://www.kristeligt-dagblad.dk/udland/tekst...,Tekst: Obamas tale,\n Læs hele præsident Barack Obamas...,"Her er en udskrift af den tale, som Barack Oba...",Tekst: Obamas tale \n Læs hele præs...,20-01-09


In [ ]:
import re
import pandas as pd

# fixing dates
def clean_date(date_str):
    match = re.search(r'\d{2}\.\d{2}\.\d{2}', date_str)
    if match:
        return match.group(0).replace('.', '-')
    return date_str

df['cleaned_date'] = df['date'].apply(clean_date)

df['standard_date'] = pd.to_datetime(df['cleaned_date'], format='%d-%m-%y')



In [ ]:
df = df.drop_duplicates(subset=['url', 'date'])
df

,search_term,url,header,subheader,content,merged_text,date
0,facial recognition,https://www.kristeligt-dagblad.dk/danmark/torsdag,"Restriktioner taber til religiøs frihed i USA,...",\n Her samler Kristeligt Dagblad no...,Morgensamling er Kristeligt Dagblads nyhedsove...,"Restriktioner taber til religiøs frihed i USA,...",2020-12-17
1,face recognition,https://www.kristeligt-dagblad.dk/udland/dokum...,Dokumentation: Læs Camerons tale om Storbritan...,\n I sin længe ventede tale om Stor...,This morning I want to talk about the future o...,Dokumentation: Læs Camerons tale om Storbritan...,2013-01-23
2,face recognition,https://www.kristeligt-dagblad.dk/udland/tekst...,Tekst: Obamas tale,\n Læs hele præsident Barack Obamas...,"Her er en udskrift af den tale, som Barack Oba...",Tekst: Obamas tale \n Læs hele præs...,2009-01-20
3,face recognition,https://www.kristeligt-dagblad.dk/kirke-tro/ta...,"Talen, som paven ikke holdt",\n Pave Benedikt XVI aflyste denne ...,It is a great joy for me to meet the community...,"Talen, som paven ikke holdt \n Pave...",2008-01-18
4,ansigtsaftryk,https://www.kristeligt-dagblad.dk/kronik/et-g%...,Et gådefuldt spejlbillede,\n Er ligklædet i Torino et håndgri...,Det er under alle omstændigheder vigtigt at hu...,Et gådefuldt spejlbillede \n Er lig...,2000-06-16
...,...,...,...,...,...,...,...
234,ansigtsgenkendelse,https://www.kristeligt-dagblad.dk/danmark/dans...,Danskere slipper for visum til USA,\n Den danske pas- sikkerhed er god...,Danmark vil kunne leve op til USA's krav om øg...,Danskere slipper for visum til USA \n ...,08-01-04
235,ansigtsaftryk,https://www.kristeligt-dagblad.dk/kronik/et-g%...,Et gådefuldt spejlbillede,\n Er ligklædet i Torino et håndgri...,Det er under alle omstændigheder vigtigt at hu...,Et gådefuldt spejlbillede \n Er lig...,16-06-00
236,face recognition,https://www.kristeligt-dagblad.dk/udland/dokum...,Dokumentation: Læs Camerons tale om Storbritan...,\n I sin længe ventede tale om Stor...,This morning I want to talk about the future o...,Dokumentation: Læs Camerons tale om Storbritan...,23-01-13
237,face recognition,https://www.kristeligt-dagblad.dk/udland/tekst...,Tekst: Obamas tale,\n Læs hele præsident Barack Obamas...,"Her er en udskrift af den tale, som Barack Oba...",Tekst: Obamas tale \n Læs hele præs...,20-01-09


In [ ]:
df = df.rename(columns={'merged_text': 'merged_content'})

In [ ]:
from google.colab import files

df.to_csv('KD.csv', index=False)
files.download('KD.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>